# Workflow for Build Workout Dashboard

In [1]:
from utilities import *

# Load the pyproject.toml file
with open("pyproject.toml", "r") as f:
    config = toml.load(f)

dbConfig = config['tool']['project']['database']

input_file = config['tool']['project']['input_file']


In [3]:
## DISPLAY DATABASES VISIBLE TO THIS USER:

connection = mysql.connector.connect(**dbConfig)
cursor = connection.cursor()

# Execute the SHOW DATABASES command
cursor.execute("SHOW DATABASES")

# Fetch all results
databases = cursor.fetchall()

# Print the databases
print("Databases visible to your user:")
for db in databases:
    print(db[0])



Databases visible to your user:
information_schema
TESTDB
amazon_product_reviews
mysql
performance_schema
running
sweat
sys


# Main function

## Step 1 - Ingest the most recent workout_history file


In [ ]:
### Read CSV file
df = pd.read_csv('user2632022_workout_history.csv')

### Clean data (drop unnecessary columns)
df = clean_data(df)

# Enrich data (for now, just extract workoutID_
df = enrich_data(df)

## Step 2 - Create whole database from file (for now)

In [ ]:
try:
    # Establish MySQL connection
    connection = mysql.connector.connect(**config)

    if connection.is_connected():
        cursor = connection.cursor()

        # If db exists, drop it
        # cursor.execute("DROP DATABASE IF EXISTS sweat")
        
        # Setup database and table
        setup_database(cursor)

        # Insert data
        rows_affected = insert_data_NEW(cursor, df)

        # Commit changes
        connection.commit()

        print(f"Data import completed. {rows_affected} rows were inserted.")

        # Get the last inserted id
        cursor.execute("SELECT LAST_INSERT_ID()")
        last_id = cursor.fetchone()[0]
        print(f"The last inserted ID was: {last_id}")

        # Get the total number of rows in the table
        cursor.execute("SELECT COUNT(*) FROM workout_summary")
        total_rows = cursor.fetchone()[0]
        print(f"Total rows in the table after insert: {total_rows}")

except Error as e:
    print(f"Error: {e}")
    print(f"Error Code: {e.errno}")
    print(f"SQLSTATE: {e.sqlstate}")
    print(f"Message: {e.msg}")
    
finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")
        

## Use connection

In [ ]:
# Establish MySQL connection
if not(connection.is_connected()):
    connection = mysql.connector.connect(**config)
    cursor = connection.cursor()

cursor.execute("USE sweat;")
# cursor.execute("SELECT COUNT(*) FROM workout_summary;")
cursor.execute("DESCRIBE workout_summary;")
cursor.fetchall()

## Test using pyproject.toml for project config settings

In [ ]:
# Access custom configuration under [tool.my_project]
project_config = config["tool"]["build_workout_dashboard"]
debug = project_config["debug"]
log_level = project_config["log_level"]
max_retries = project_config["max_retries"]

# Access database config
db_config = project_config["database"]
db_host = db_config["host"]
db_port = db_config["port"]
db_user = db_config["username"]
db_password = db_config["password"]

# Access API config
api_config = project_config["api"]
api_base_url = api_config["base_url"]
api_key = api_config["api_key"]

print(f"Debug Mode: {debug}")
